In [1]:
import pandas as pd
import numpy as np
import tqdm
import os

In [2]:
from keras.preprocessing import image

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import gc

In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import log_loss, accuracy_score
from sklearn.linear_model import LogisticRegression

In [5]:
from keras.applications import xception

In [6]:
def read_img(img_id, folder, size):
    """Read and resize image.
    
    Arguments
    ---------
        img_id: string
        
        folder: string
        
        size: tuple
            Target size to resize the original image into.
    
    Returns
    -------
        img: np.ndarray
            Image as numpy array.
    """
    img = image.load_img(os.path.join(folder, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

In [7]:
# All preparation things are adapted from https://www.kaggle.com/gaborfodor/dog-breed-pretrained-keras-models-lb-0-3
INPUT_SIZE = 224
SEED = 20180407
data_dir = '../data/dog_breed'
labels = pd.read_csv(os.path.join(data_dir, 'labels.csv'))
sample_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
NUM_CLASSES = labels["breed"].nunique()

In [8]:
# load images
INPUT_SIZE = 224
POOLING = 'avg'
train_folder = "../data/dog_breed/Train"
y_train = labels["breed"].values
x_train = np.zeros((labels.shape[0], INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in tqdm.tqdm(enumerate(labels['id'])):
    img = read_img(img_id, train_folder, (INPUT_SIZE, INPUT_SIZE))
    # x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = img  # it's x originally
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

10222it [01:10, 144.51it/s]

Train Images shape: (10222, 224, 224, 3) size: 1,538,697,216


In [10]:
# split training and validation set
np.random.seed(seed=SEED)
rnd = np.random.random(len(labels))
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8

# prepare train/test data
Xtr = x_train[train_idx]
Xv = x_train[valid_idx]
del x_train
gc.collect()  # clear the memory or your computer will explode lol

# prepare labels
lb = LabelBinarizer()
y_onehot = lb.fit_transform(y_train)
ytr_onehot = y_onehot[train_idx]
yv_onehot = y_onehot[valid_idx]
label_idx_dict = {label: idx for idx, label in enumerate(lb.classes_)}
idx_label_dict = {idx: label for idx, label in enumerate(lb.classes_)}
ycode = np.array([label_idx_dict[label] for label in y_train])
ytr = ycode[train_idx]
yv = ycode[valid_idx]

In [20]:
xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
train_x_bf = xception_bottleneck.predict(Xtr, batch_size=32, verbose=1)
valid_x_bf = xception_bottleneck.predict(Xv, batch_size=32, verbose=1)
print('Xception train bottleneck features shape: {} size: {:,}'.format(train_x_bf.shape, train_x_bf.size))
print('Xception valid bottleneck features shape: {} size: {:,}'.format(valid_x_bf.shape, valid_x_bf.size))

2060/2060 [==============================] - 690s   
Xception train bottleneck features shape: (8162, 2048) size: 16,715,776
Xception valid bottleneck features shape: (2060, 2048) size: 4,218,880


In [21]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg.fit(train_x_bf, ytr)
valid_probs = logreg.predict_proba(valid_x_bf)
valid_preds = logreg.predict(valid_x_bf)
print('Validation Xception LogLoss {}'.format(log_loss(yv_onehot, valid_probs)))
print('Validation Xception Accuracy {}'.format(accuracy_score(yv, valid_preds)))

Validation Xception LogLoss 5.52039454292717
Validation Xception Accuracy 0.04563106796116505


In [22]:
# # the validation score sucks so I give up making prediction for test data
# test_folder = "../data/dog_breed/Test"
# x_test = np.zeros((sample_submission.shape[0], INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
# for i, img_id in tqdm.tqdm(enumerate(sample_submission['id'])):
#     img = read_img(img_id, test_folder, (INPUT_SIZE, INPUT_SIZE))
#     # x = preprocess_input(np.expand_dims(img.copy(), axis=0))
#     x_test[i] = img  # it's x originally
# print('Test Images shape: {} size: {:,}'.format(x_test.shape, x_test.size))

In [23]:
# test_x_bf = xception_bottleneck.predict(x_test, batch_size=32, verbose=1)
# test_probs = logreg.predict_proba(test_x_bf)